# 전이 학습(Transfer Learning)
- 전이 학습은 기존에 학습된 모델 또는 특징 추출기를 사용하여 새로운 작업에 대한 모델을 초기화하거나 초기화한 모델을 추가로 학습시키는 학습 방법
  - 특징 추출(Feature Extraction)
    - 기존의 학습된 conv층은 유지하여 일반적인 특징을 캡처. 따라서 새로운 데이터셋에 맞게 분류기만 학습
  - 미세 조정(Fine-tuning)
    - 사전 학습된 모델의 일부 또는 전체 레이어를 새로운 작업에 맞게 추가 학습
    - 모델의 상위 레이어는 새로운 작업에 더 적응되도록 새로운 데이터셋으로 학습.
    - 일반적으로 학습률(learning rate)을 낮추고, 이전에 학습된 가중치에 더 큰 패널티를 부여하여 오버피팅을 방지.

In [ ]:
# 환경 변수
cfg = {
    'train_size': (512, 512),  # 이미지 크기
    'dict_label': {
        '02': {'00': 0, '03': 1, '04': 2},  # 고추마일드, 고추점무늬
        '05': {'00': 3, '09': 4, '10': 5},  # 상추균핵병, 상추노균병
        '11': {'00': 6, '18': 7, '19': 8},  # 토마토황화잎, 토마토잎곰팡이
    }
}
num_classes = 3 + 3 + 3  # 고추마일드, 고추점무늬, 상추균핵병, 상추노균병, 토마토황화잎, 토마토잎곰팡이

### 미세 조정(Fine-tuning)


In [ ]:
import cv2
import numpy as np
import zipfile
from google.colab.patches import cv2_imshow
import tensorflow as tf
import seaborn as sns # 데이터 시각화
# 서로 다른 2개의 모델을 합치기 때문에 순차적은 아니다.
from tensorflow.keras.models import Model
# 전이 학습을 위해 모델 합치기위한 객체
from tensorflow.keras.layers import Dense, Input,GlobalAveragePooling2D, Dropout
# 이미지 전처리를 도와주는 객체( 테스트 데이터, 검증 데이터 분할, 이미지 늘리기 등 )

tf.__version__

#### Pre-trained Network(사전 학습 모델) Loading
- ResNet50과 imagenet을 사용한다.
- Network
  - ResNet : https://arxiv.org/pdf/1512.03385.pdf
  - keras에서 제공하는 사전학습 가능한 모델들
    - documentation : https://keras.io/api/applications/
- weights
  - imagenet : https://www.image-net.org/
  - coco : https://cocodataset.org/#home

In [ ]:
# 사전 학습된 ResNet50 모델을 가져온다. (이미지넷 데이터셋으로 사전 학습된 가중치를 사용한다.)
# include_top = False는 밀집층을 포함하지 않고 모델의 최상위 레이어를 가져온다.
# 이는 사용자가 새로운 작업에 맞게 자신만의 분류기를 추가할 수 있게 한다.
# 각 사전 학습된 모델은 다양한 입력 크기를 지원한다. 사용자는 자신의 데이터셋에 맞는 입력 크기를 선택할 수 있다.

base_model = tf.keras.applications.ResNet50(
    weights='imagenet',  # ImageNet 데이터셋으로 사전 학습된 가중치를 사용.
    include_top=False,   # 최상위 밀집층을 포함하지 않습니다.
    input_tensor=Input(shape=(512, 512, 3))  # 네트워크의 입력 크기를 지정.
)

In [ ]:
base_model.summary()

In [ ]:
len(base_model.layers)

In [ ]:
base_model.trainable = True

* ResNet50 모델의 각 층은 입력에 대해 다른 수준의 추상화를 학습한다.
  - 일반적으로 모델의 하위 층은 저수준의 기능(예: 선, 모서리)을 학습하고 상위 층은 더 추상적이고 고수준의 기능(예: 개, 자동차)을 학습한다.
  
  - 따라서 전체 네트워크를 재학습하는 대신 상위 층 몇 개만을 재학습하는 것이 일반적이고 이것은 더 적은 데이터로 더 빠르게 모델을 학습할 수 있도록 해준다.

  - "fine_tuning_at = 120" 설정으로, 모델의 하위층인 120층 미만은 저수준 특징(선, 모서리 등)을 그대로 사용하면서 미세 조정되지 않는다. 반면, 120 이상의 상위층은 고수준 특징(개, 자동차 등)을 학습하여 새로운 데이터셋에 잘 적응하도록 돕는다.

In [ ]:
fine_tuning_at = 140

# 0층부터 120층까지 layer을 학습 불가능하도록 가중치 고정
# 앞에 층은 보통 이미지의 저차원 수준의 특징을 나타내기 때문에 학습이 불 필요하다.
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

In [ ]:
# output은 모델의 마지막 출력 정보를 가지고있다.(conv층에 의해 생성, 16,16,2048형태로 데이터 출)
base_model.output

In [ ]:
# output은 모델의 마지막 출력 정보를 가지고있다.(conv층에 의해 생성, 16,16,2048형태로 데이터 출)
head_model = base_model.output

# 출력된 행렬을 평균값을 이용하여 벡터로 변환해준다.
# 평탄화 함수를 사용할 수 있지만 GlobalAveragePooling2D()함수를 통해 평탄화 할수도 있다
head_model = GlobalAveragePooling2D()(head_model)

# 은닉층 1 -> 은닉층의 뉴런 수는 입력층(2048)과 출력층(클래스 수)의 평균값인 1025로 설정합니다.
head_model = Dense(units = 1025, activation = 'relu')(head_model)

# Dropout을 통해 20% 뉴런들을 비활성화하여 과적합 방지
head_model = Dropout(rate = 0.2)(head_model)

# 은닉층2
head_model = Dense(units = 1025, activation = 'relu')(head_model)

# Dropout을 통해 20% 뉴런들을 비활성화하여 과적합 방지
head_model = Dropout(rate = 0.2)(head_model)

# softmax 함수는 다중 클래스 분류 문제에서 사용, 각 클래스에 속할 확률을 계산 - > 모든 요소의 합은 1
# 크로스 엔트로피와 같은 일부 손실 함수 적용 가능
head_model = Dense(units=num_classes, activation='softmax')(head_model)  # num_classes는 클래스의 개수

In [ ]:
# 두 신경망 연결하기 (Model (inputs = 입력에 사용될 신경망, outputs = 출력할 신경망))
network = Model(inputs = base_model.input, outputs= head_model)

1028.5

In [ ]:
# 주의점 : 요약의 평탄화의 입력shape을 보면 2048이다. 그냥 평탄화 함수를 썼으면 16*16*2048의 입력층이 나올것이다.
# 하지만 global_average_pooling2d를 사용함으로 입력층에 들어갈 데이터크기를 대폭 감소시킬수 있다.
network.summary()

### 학습

#### 학습 진행

In [ ]:
# GPU 확인
tf.config.list_physical_devices('GPU')

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

network.compile(optimizer=Adam(learning_rate=0.0008),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# loss가 제일 낮은 모델 가중치만 저장한다.
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
# 에포크가 진행됨에도 로스가 줄어들지 않다면 학습을 조기 종료한다.
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

history = network.fit(train_dataset,
            validation_data=valid_dataset,
            epochs=100,
            callbacks=[checkpoint])

Epoch 1/100
94/94 [==============================] - ETA: 0s - loss: 3.0238 - accuracy: 0.1372

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94/94 [==============================] - 247s 2s/step - loss: 3.0238 - accuracy: 0.1372 - val_loss: 2.2396 - val_accuracy: 0.0752
Epoch 2/100
94/94 [==============================] - 233s 2s/step - loss: 1.9173 - accuracy: 0.3076 - val_loss: 8.6828 - val_accuracy: 0.0391
Epoch 3/100
94/94 [==============================] - 234s 2s/step - loss: 1.9164 - accuracy: 0.3991 - val_loss: 13.7704 - val_accuracy: 0.0391
Epoch 4/100
94/94 [==============================] - 232s 2s/step - loss: 1.7875 - accuracy: 0.2412 - val_loss: 9.5443 - val_accuracy: 0.0391
Epoch 5/100
94/94 [==============================] - 234s 2s/step - loss: 1.6880 - accuracy: 0.3780 - val_loss: 7.6881 - val_accuracy: 0.0391
Epoch 6/100
94/94 [==============================] - 234s 2s/step - loss: 1.6061 - accuracy: 0.3896 - val_loss: 9.6400 - val_accuracy: 0.0391
Epoch 7/100
94/94 [==============================] - 233s 2s/step - loss: 1.8470 - accuracy: 0.5107 - val_loss: 4.0379 - val_accuracy: 0.0512
Epoch 8/100
94/94

#### 학습된 모델 저장 및 불러오기
- save로 모델저장
  - 학습 결과가 폴더로 생성된다.
    - saved_model.pd 는 신경망 모델의 구조를 나타낸다
    - variables는 가중치를 나타낸다
  - 모델을 불러올때도 폴더 전체를 불러와야한다.
- 가중치 및 구조를 따로 저장
  - 모델의 구조를 json으로 저장
    - model.to_json()함수 사용
  - 가중치는 h5(HDF5)파일로 저장
    - model.save_weights("파일경로")함수 사용
- keras 모델을 사용할때 가중치와 구조를 한파일로 저장
  - model.save("model.h5") 함수 사용

In [ ]:
# 모델 저장 ( 가중치 및 구조가 같이 저장됨 )
network.save("/content/saved_model")

In [ ]:
# 모델 불러오기
loaded_ = tf.keras.models.load_model("/content/best_model.h5")

In [ ]:
loaded_.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 518, 518, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 256, 256, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 256, 256, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

#### 학습 결과 분석 (시각화)
- 에포크 마다 정확도 그래프 보기
- 에포크 마다 손실 그래프 보기

In [ ]:
import matplotlib.pyplot as plt
# 학습 및 검증 손실 값을 플로팅
plt.figure(figsize=(12, 4))

# 학습 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 학습 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()